In [0]:
# Imports

import pandas as pd
import os
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Retrieve the CONFIDENTIAL patient dataset with test results

# You must mount your drive if you have access to the ***CONFIDENTIAL PATIENT TEST RESULTS***
test_result_array = []
test_results_path = "./drive/My Drive/patient-test-results/pcr-result"

for dirpath, dnames, fnames in os.walk(test_results_path):
    for f in fnames:
        if f.endswith(".csv"):
            print(f)
            file_path = os.path.join(test_results_path, f)
            test_result_array += [pd.read_csv(file_path)] # Converts to a dataframe
   




In [0]:
# Read the Patient Raw Data 

# Retrieve the JSON
!wget --user <USER> --password <PASSWORD> <LINK>



--2020-04-16 11:37:35--  https://detect-now.org/pakistan/url/org/admin_access_data
Resolving detect-now.org (detect-now.org)... 52.178.92.96
Connecting to detect-now.org (detect-now.org)|52.178.92.96|:443... connected.
HTTP request sent, awaiting response... 401 UNAUTHORIZED
Authentication selected: Basic realm="Login Required"
Reusing existing connection to detect-now.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 34393 (34K) [application/json]
Saving to: ‘admin_access_data.2’

admin_access_data.2 100%[===================>]  33.59K  --.-KB/s    in 0.09s   

2020-04-16 11:37:35 (355 KB/s) - ‘admin_access_data.2’ saved [34393/34393]



In [0]:
# Convert to a df

json_df = pd.read_json(r'/content/admin_access_data')
json_df.head()


In [0]:
#TO DO: iterate through all the files provided by the hospital for matching


daily_df_hospital = test_result_array[1]
daily_df_hospital.head()

In [0]:
# Code to count the number of matches!

#TO DO: iterate through all the files provided by the hospital for matching

counter = 0
match_counter = 0

for patient_id in json_df['patientID']:
  counter_1 = 0
  for patient_id_2 in daily_df_hospital['COVID DATA 13/04/2020']: #Change the date column 
    patient_id = str(patient_id).replace('-' , '')
    patient_id_2 = str(patient_id_2).replace('-' , '')
    if patient_id == patient_id_2:
      json_df['corona_test'][counter] = daily_df_hospital['Unnamed: 3'][counter_1] # Change the result column 
      match_counter += 1
    counter_1 += 1
  counter += 1
 




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [0]:
# Anonymize Output


json_df = json_df.drop(columns='patient_id')
json_df = json_df.drop(columns="patientID")

json_df.head()

In [0]:
# Count the Percentage of Correctly Matched
#counter = 0
#for result in deleted_PatientID['corona_test']:
# if result[-1].isupper():
#    counter += 1



print('The percentage of correctly mapped raw files is: ', round(match_counter/len(json_df['corona_test']) * 100. ,2) , '%') 

The percentage of correctly mapped raw files is:  8.54 %


In [0]:
json_df.loc[(json_df.corona_test == 'NOT DETECTED'), 'corona_test'] = 'NEGATIVE'
json_df.loc[(json_df.corona_test == 'Negative'), 'corona_test'] = 'NEGATIVE'
json_df.loc[(json_df.corona_test == 'DETECTED'), 'corona_test'] = 'POSTIVE'
json_df.loc[(json_df.corona_test == 'Positive'), 'corona_test'] = 'POSTIVE'



In [0]:
# Store as a .csv
filename = 'output-04-15-2020-1.csv'
json_df.to_csv('/content/drive/My Drive/patient-test-results/curated-outputs/' + filename )